## WSC Quiz Coin
#### 03 Smart Contracts Testing
##### Samuele Dell'Oca (samuele.delloca@supsi.ch)

The entire notebook is predisposed in such a way that the cells have to be executed in a sequential way for replicating the desired execution. In case some cells need to be skipped, there will be a proper indication.
Be sure that the smart contracts are still valid before testing them.

## Notebook Setup
Taken from the course examples. It loads:
* The functions in `algo_util.py`
* The accounts MyAlgo, Alice and Bob
* The Purestake credentials

In [1]:
# Loading shared code and credentials
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 5 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.transaction import LogicSig, LogicSigTransaction, LogicSigAccount
import algosdk.error
import json
import base64

from pyteal import *
import random
import hashlib

In [3]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

### Using the created Smart Contracts

First of all we have to set the address of the deployed contracts. Then it will be possible to use them.

In [ ]:
asset_id = 175523481
asset_decimals = 2
dispense_hash = "YVGRKP255O2MRGK4P6UVTQU2W4DNZSOMI4PXPLEQSN5WV7GJEIVDHE4QGQ"
dispense_result = "CCAEAQCL7fyKDAQkJBIxAYHoBw4QMgQiEkAAWzIEgQISMwAQIhIQMwARIxIQMwEQJRIQMwERgZmN2VMSEIHAhD0zAAgLIxIQMwAHMwEAEhAzAAAzARQSEDMAIDIDEhAzAAkyAxIQMwEgMgMSEDMBFTIDEhBCABoyBCISMRAlEhAxEiMSEDEgMgMSEDEJMgMSEBBD"
country_quiz_hash = "VRBJLYDADJWW6NF7NEDTG6343LGUYZFPSCGVL374ZXFZEYHGDDPISQXI7M"
country_quiz_result = "CCAFAQQAut/vswGZjdlTJgEgmLUFmOFjnZBD7fj7YFWjJ9+GwqzlS0hj1j21yzBGIgglJRIxAYHoBw4QMgQiEkAArjECgZnx9w0MQAAnMwAQIxIzABEhBBIQMwAUKBIQMwEQIhIQMwERJBIQMwEHKBIQQgCWMgSBAhIzABAjEhAzABEhBBIQMwEQIhIQMwERJBIQMwASgWQSEDMABQGAICJ1WDGW15FAWJKqyg2HdDyHLz/AzzMIpsPvglKJGKqKEhAzABQzAQASEDMAADMBBxIQMwAgMgMSEDMACTIDEhAzASAyAxIQMwEVMgMSEEIAGjIEIhIxECMSEDESJBIQMSAyAxIQMQkyAxIQEEM="
vending_hash = "2OD3S53WDKSYJIFPDHC5SJIPZBDYJ2BS4UJJP7QYSS3OGPPVRGPO5DL52I"
vending_result = "CCAHAWTAhD3l/I5uAAQKJSUSMQGB6AcOEDIEIhJAAJ4yBIECEjMAECISEDMAESEEEhAzARAhBRIQMwERgZmN2VMSEDMACIGAreIEDEAAYTMACIGA4esXDEAAQSMzAAghBgqBCAsLJDMBEgsSEDMABzMBABIQMwAAMwEUEhAzACAyAxIQMwAJMgMSEDMBIDIDEhAzARUyAxIQQgA+IzMACCEGCoEJCwskMwESCxJC/7wjMwAICyQzARILEkL/rjIEIhIxECEFEhAxEiEEEhAxIDIDEhAxCTIDEhAQQw=="

### Classical attacks
Anti-theft, anti-money laundering, fee attacks, rekey attacks and other sort of attacks are managed in the relative programs. So I will check only for problems or attacks relative to the specific implementation, such as sending wrong answers or requesting wrong quantities from the Vending contract.

### Dispenser

In [ ]:
#  Bob (a user) needs to opt into the Quiz Coin before participating.

sp = algod_client.suggested_params()
amt = int(0)

txn = AssetTransferTxn(
    sender=Bob['public'],
    sp=sp,
    receiver=Bob['public'],
    amt=amt,
    index=asset_id
)

#  opt-in signing and sending
stxn = txn.sign(Bob['private'])
txid = algod_client.send_transaction(stxn)

txinfo = wait_for_confirmation(algod_client, txid)

In [1]:
#  the Dispenser is ready to use. Bob gets 5 Quiz Coins.

sp = algod_client.suggested_params()

#  ALGO payment from Bob to Dispenser
amt_1 = int(1.0 * 1e6)  # WON'T WORK!
txn_1 = PaymentTxn(Bob['public'], sp, dispense_hash, amt_1)

#  Quiz Coin asset transfer from Dispenser to Bob
amt_2 = int(5 * 10**asset_decimals)
txn_2 = AssetTransferTxn(dispense_hash, sp, Bob['public'], amt_2, asset_id)

#  transaction group creation
gid = transaction.calculate_group_id([txn_1, txn_2])
txn_1.group = gid  # add group_id to each transaction
txn_2.group = gid

#  Bob signs the first transaction, Smart signature will authorize the second one
stxn_1 = txn_1.sign(Bob['private'])
encodedProg = dispense_result.encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn_2 = LogicSigTransaction(txn_2, lsig)

#  transactions sending
signed_group = [stxn_1, stxn_2]
txid = algod_client.send_transactions(signed_group)

txinfo = wait_for_confirmation(algod_client, txid)

NameError: name 'algod_client' is not defined

### Question and answer

In [ ]:
#  Quiz answer submission

sp = algod_client.suggested_params()

my_note = 'Zurich'  # WON'T WORK!
note = my_note.encode()

#  Quiz Coin asset transfer from Bob to Smart Signature
amt_1 = int(1.0 * 1e2)
txn_1 = AssetTransferTxn(Bob['public'], sp, country_quiz_hash, amt_1, asset_id, note=note)

#  Algo prize transfer from Smart Sigature to Bob
# amt_2 = int(50.0 * 1e6)
amt_2 = int(1.0 * 1e6)  # it should be 50, I've used lower quantities for testing
txn_2 = PaymentTxn(country_quiz_hash, sp, Bob['public'], amt_2)

#  transaction group creation, signing and sending
gid = transaction.calculate_group_id([txn_1, txn_2])
txn_1.group = gid
txn_2.group = gid

stxn_1 = txn_1.sign(Bob['private'])
encodedProg = country_quiz_result.encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn_2 = LogicSigTransaction(txn_2, lsig)

signed_group = [stxn_1, stxn_2]
txid = algod_client.send_transactions(signed_group)

txinfo = wait_for_confirmation(algod_client, txid)

In [ ]:
#  money retrieving from Quiz Company after quiz closure (1 week). The company retrieves both the Quiz Coins (first transaction) and the ALGOs (second transaction).

# WON'T WORK BEFORE 1 WEEK

sp = algod_client.suggested_params()

#  Quiz Coins retrieving from Smart Signature
amt_1 = int(1.0 * 1e2) #  it should be set based on the amount of Quiz Coins
txn_1 = AssetTransferTxn(country_quiz_hash, sp, MyAlgo['public'], amt_1, asset_id)

#  ALGOs retrieving from Smart Signature
amt_2 = int(0.1 * 1e6)  #  it should be set based on the amount of ALGOs
txn_2 = PaymentTxn(country_quiz_hash, sp, MyAlgo['public'], amt_2)

#  transaction group creation, signing and sending
gid = transaction.calculate_group_id([txn_1, txn_2])
txn_1.group = gid
txn_2.group = gid

encodedProg = country_quiz_result.encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn_1 = LogicSigTransaction(txn_1, lsig)
stxn_2 = LogicSigTransaction(txn_2, lsig)

signed_group = [stxn_1, stxn_2]
txid = algod_client.send_transactions(signed_group)

txinfo = wait_for_confirmation(algod_client, txid)

### Vending

In [ ]:
##  using the selling contract

sp = algod_client.suggested_params()

#  payment transaction in ALGO from Bob to Vending
amt_1 = int(5.0 * 1e6)
txn_1 = PaymentTxn(Bob['public'], sp, vending_hash, amt_1)

#  asset transfer transaction from Vending to Bob
amt_2 = int(10.0 * 1e2)  # WON'T WORK
txn_2 = AssetTransferTxn(vending_hash, sp, Bob['public'], amt_2, asset_id)

#  transaction group creation, signing and sending
gid = transaction.calculate_group_id([txn_1, txn_2])
txn_1.group = gid
txn_2.group = gid

stxn_1 = txn_1.sign(Bob['private'])
encodedProg = vending_result.encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn_2 = LogicSigTransaction(txn_2, lsig)

signed_group = [stxn_1, stxn_2]
txid = algod_client.send_transactions(signed_group)

txinfo = wait_for_confirmation(algod_client, txid)

In [ ]:
asset_id = 175523481